In [2]:
import eumdac
import datetime
import shutil
from tqdm import tqdm
import os
import zipfile

In [3]:
# Establish directory locations
parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir    = os.path.join(parent_dir, 'Output/sat_plot')
NEODASS_dir   = os.path.join(satellite_dir, 'NEODASS')
floats_dir    = os.path.join(parent_dir, 'Data/Floats')
sentinel_dir  = os.path.join(satellite_dir, 'SENTINEL')
sen3_dirs     = [os.path.join(sentinel_dir, d) for d in os.listdir(sentinel_dir) if '.SEN3' in d]

In [4]:
consumer_key = '9dcHtMpRngCEnHBtZX4iOZL7s5Qa'
consumer_secret = 'FfJCsfRfPPOThUvOYfBEFkPzxCwa'
credentials = (consumer_key, consumer_secret)
token = eumdac.AccessToken(credentials)
print(f"This token '{token}' expires {token.expiration}")

datastore = eumdac.DataStore(token)
for collection in datastore.collections:
    print(collection)
selected_collection = datastore.get_collection('EO:EUM:DAT:0410')

This token '722a85dc-1bca-3cfa-b42c-1cd89b8e2479' expires 2024-06-17 15:49:28.150777
EO:EUM:DAT:MSG:CLM-IODC
EO:EUM:DAT:0394
EO:EUM:DAT:0412
EO:EUM:DAT:0237
EO:EUM:DAT:0241
EO:EUM:DAT:0857
EO:EUM:DAT:0862
EO:EUM:DAT:0882
EO:EUM:DAT:METOP:ASCSZF1B
EO:EUM:DAT:METOP:ASCSZR1B
EO:EUM:DAT:MSG:MSG15-RSS
EO:EUM:DAT:0855
EO:EUM:DAT:0836
EO:EUM:CM:METOP:ASCSZFR02
EO:EUM:DAT:0081
EO:EUM:DAT:0398
EO:EUM:DAT:0405
EO:EUM:DAT:0406
EO:EUM:DAT:0410
EO:EUM:DAT:0413
EO:EUM:DAT:0415
EO:EUM:DAT:0416
EO:EUM:DAT:0239
EO:EUM:DAT:0301
EO:EUM:DAT:0617
EO:EUM:DAT:0880
EO:EUM:DAT:0856
EO:EUM:DAT:METOP:ASCSZO1B
EO:EUM:DAT:METOP:GOMEL1
EO:EUM:DAT:0647
EO:EUM:DAT:0851
EO:EUM:DAT:0835
EO:EUM:DAT:0833
EO:EUM:DAT:0858
EO:EUM:DAT:0859
EO:EUM:DAT:0839
EO:EUM:DAT:0841
EO:EUM:DAT:0834
EO:EUM:DAT:0142
EO:EUM:DAT:0238
EO:EUM:DAT:0374
EO:EUM:DAT:0601
EO:EUM:DAT:0556
EO:EUM:DAT:0557
EO:EUM:DAT:0576
EO:EUM:DAT:0579
EO:EUM:DAT:0581
EO:EUM:DAT:0583
EO:EUM:DAT:0300
EO:EUM:DAT:0407
EO:EUM:DAT:0411
EO:EUM:DAT:0417
EO:EUM:DAT:0274
EO

In [5]:
# Add vertices for polygon, wrapping back to the start point.
min_lat = 55.5
max_lat = 64.5
min_lon = -30
max_lon = -15
geometry = [[min_lon, min_lat],
            [min_lon, max_lat],
            [max_lon, max_lat],
            [max_lon, min_lat],
            [min_lon, min_lat]]

# Set sensing start and end time
start = datetime.datetime(2024, 6, 17)
end = datetime.datetime(2024, 6, 18)

# Retrieve datasets that match our filter
products = selected_collection.search(
           geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in geometry])),
           dtstart=start, 
           dtend=end)
    
print(f'Found Datasets: {products.total_results} datasets for the given time range')
for product in products:
    print(str(product))

Found Datasets: 1 datasets for the given time range
S3A_OL_1_ERR____20240617T110032_20240617T114457_20240617T132442_2665_113_308______MAR_O_NR_002.SEN3


In [6]:
# Function to download files
def download_files(products, download_dir):
    print(f'Found Datasets: {products.total_results} datasets for the given time range')
    for product in products:
        print(str(product))

    for product in products:
        with product.open() as fsrc:
            # Get the total length of the file from the response headers
            total_length = int(fsrc.info().get('Content-Length', 0))

            # Define the destination file path
            dest_path = os.path.join(download_dir, os.path.basename(fsrc.name))

            # Open the destination file
            with open(dest_path, mode='wb') as fdst, \
                 tqdm(total=total_length, unit='B', unit_scale=True, desc=f"Downloading {product}") as pbar:
                
                # Define a chunk size for reading the data
                chunk_size = 1024
                
                # Read the file in chunks and write to the destination file
                for chunk in iter(lambda: fsrc.read(chunk_size), b''):
                    if chunk:  # filter out keep-alive new chunks
                        fdst.write(chunk)
                        pbar.update(len(chunk))
        
        print(f'Download of product {product} finished.')

    print('All downloads are finished.')

# Function to unzip files
def unzip_files(zip_dir):
    for item in os.listdir(zip_dir):
        if item.endswith('.zip'):
            zip_path = os.path.join(zip_dir, item)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(zip_dir)
            os.remove(zip_path)  # Optionally remove the zip file after extraction
            print(f'Unzipped {item}')

# Example usage
# Assuming `products` is already defined and contains the products to download7

download_files(products, sentinel_dir)
unzip_files(sentinel_dir)

Found Datasets: 1 datasets for the given time range
S3A_OL_1_ERR____20240617T110032_20240617T114457_20240617T132442_2665_113_308______MAR_O_NR_002.SEN3


Download of product S3A_OL_1_ERR____20240617T110032_20240617T114457_20240617T132442_2665_113_308______MAR_O_NR_002.SEN3 finished.
All downloads are finished.
Unzipped S3A_OL_1_ERR____20240617T110032_20240617T114457_20240617T132442_2665_113_308______MAR_O_NR_002.SEN3.zip
